# Training deBERTa (base version) classifier for sentimental analysis of Reddit data
Training a deep learning model (deBERTa) that is based on the transformer architecture as part of the BERT encoder, which lies at the heart of the model. The model and pre-trained weights taken from [hugging face](https://huggingface.co/microsoft/deberta-v3-base). deBerta is generally proposed in [this paper](https://arxiv.org/abs/2006.03654). Here, we take their second smallest model as listed [here](https://huggingface.co/microsoft/deberta-v3-small).

Results:
- f1 score   macro avg: 0.74
- f1 score  weighted avg: 0.79

Note (from [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)):
- F1 = 2 * (precision * recall) / (precision + recall)
- 'micro': Calculate metrics globally by counting the total true positives, false negatives and false positives.
- 'macro': Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

Also varied hyperparameters with the following results:
- learning rate/10 (0.73, 0.78)

And only trained weights of the classifier, which failed, possibly too few weights available?!
- *finetuning* (couldn't learn)

Model weights:\
Weights need to be downloaded from the [repo of microsoft/deberta-v3-base](https://huggingface.co/microsoft/deberta-v3-base/tree/main) and put in the folder of execution to use them.

Data:\
Data is downloaded when the notebook is executed from hugging face.

Generally, model seems rather memory bound and larger batchsizes result in memory errors. Maybe parallelization not optimized for memory in current configuration...

In [ ]:
# based on https://www.kaggle.com/code/tanlikesmath/feedback-prize-effectiveness-eda-deberta-baseline/notebook
# transformers taken from https://huggingface.co/microsoft/deberta-v3-small

# changed to larger model here: https://huggingface.co/microsoft/deberta-v3-base

In [ ]:
# allows update of external libraries without need to reload package
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import utils

import torch

import datasets

import skmultilearn.model_selection.iterative_stratification

import sklearn.metrics

import transformers

## Set all gpus to available gpus (not required)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [ ]:
!echo $CUDA_VISIBLE_DEVICES

In [ ]:
torch.cuda.empty_cache()  # empties gpu memory, may be required when interrupting training due bugs/user input

In [ ]:
model_nm = (
    "/p/project/deepacf/maelstrom/ehlert1/deberta-v3-base"  # model repo downloaded from hugging face see link above
)

In [ ]:
!ls /p/project/deepacf/maelstrom/ehlert1/deberta-v3-small

## Tokenize data with same tokenizer used for pre-training the model

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_nm)

In [ ]:
separator_token = tokenizer.sep_token
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

## Load data

In [ ]:
num_labels = 28
emotions = datasets.load_dataset("go_emotions", "simplified")
df_raw = pd.concat(
    [
        emotions.data["train"].table.to_pandas(),
        emotions.data["validation"].table.to_pandas(),
        emotions.data["test"].table.to_pandas(),
    ]
)
y_raw = utils.convert_df_labels(df_raw, num_labels)
df_unique = utils.remove_ambiguous_data(df_raw, y_raw)
# updated data frame shape, therefore need to recompute y labels
y_unique = utils.convert_df_labels(df_unique, num_labels)
# explanation for iterative stratification of labels http://videolectures.net/ecmlpkdd2011_tsoumakas_stratification/?q=stratification%20multi%20label
(
    indices_train,
    y_train,
    indices_test,
    y_test,
) = skmultilearn.model_selection.iterative_stratification.iterative_train_test_split(
    np.arange(df_unique.shape[0]).reshape(-1, 1), y_unique, 0.1
)
indices_train = indices_train[:, 0]
indices_test = indices_test[:, 0]

In [ ]:
y_binarized = utils.binarize_labels_torch(y_unique)

In [ ]:
df_unique["label"] = y_binarized

In [ ]:
df_reduced = df_unique.rename(columns={"text": "inputs"})
df_reduced = df_reduced.drop(columns=["id", "labels"])

In [ ]:
_dataset = datasets.Dataset.from_pandas(df_reduced)

In [ ]:
def tok_func(x):
    return tokenizer(x["inputs"], truncation=True)

In [ ]:
tokenized_dataset = _dataset.map(tok_func, batched=True, remove_columns="inputs")

### Convert dataset to object readable by `transformers.trainer`

In [ ]:
dataset_training = datasets.DatasetDict(
    {
        "train": tokenized_dataset.select(indices_train),
        "test": tokenized_dataset.select(indices_test),
    }
)

In [ ]:
def get_dds(df, train=True):
    ds = datasets.Dataset.from_pandas(df)
    to_remove = ["label"]
    tok_ds = ds.map(tok_func, batched=True, remove_columns=to_remove)
    if train:
        return datasets.DatasetDict(
            {
                "train": tok_ds.select(indices_train),
                "test": tok_ds.select(indices_test),
            }
        )
    else:
        return tok_ds

In [ ]:
def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists.

    Defined in :numref:`sec_use_gpu`"""
    devices = [torch.device(f"cuda:{i}") for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device("cpu")]

## Setting hyperparameters  

In [ ]:
learning_rate = 3e-05
train_batch_size = 4
eval_batch_size = 4
weight_decay = 0
epochs = 3

In [ ]:
def score(preds):
    return {
        "log loss": sklearn.metrics.log_loss(
            preds.label_ids,
            torch.nn.functional.softmax(torch.Tensor(preds.predictions)),
        )
    }

## Defining trainer object

In [ ]:
# lr_scheduler_type='cosine' --> linear
num_labels = 2


def get_trainer(dds, tokenizer, finetuning=True):
    """gets trainer object, finetuning fixes parameters in the model to and only sets parameters of the classifier part to trainable"""
    args = transformers.TrainingArguments(
        "/p/project/deepacf/maelstrom/ehlert1/output_RedditSentimentMultiLabelClassificationTransformerBaseline/",
        learning_rate=learning_rate,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
        fp16=True,
        evaluation_strategy="epoch",
        per_device_train_batch_size=train_batch_size,
        per_device_eval_batch_size=eval_batch_size,
        num_train_epochs=epochs,
        weight_decay=weight_decay,
        report_to="none",
    )
    model = transformers.AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=2)
    if finetuning:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.classifier.parameters():
            param.requires_grad = True
    return transformers.Trainer(
        model,
        args,
        train_dataset=dds["train"],
        eval_dataset=dds["test"],
        tokenizer=tokenizer,
        compute_metrics=score,
    )

In [ ]:
try_all_gpus()

In [ ]:
trainer = get_trainer(dataset_training, tokenizer, finetuning=False)

## Training

In [ ]:
trainer.train()

## Check trainable and non-trainable parameters 

In [ ]:
print(
    f"total number of parameters: {sum([p.numel() for p in trainer.model.parameters()])}, trainable parameters: {sum([p.numel() for p in trainer.model.parameters() if p.requires_grad])}"
)

## Validation 

In [ ]:
test_ds = get_dds(df_reduced.iloc[indices_test], train=False)

In [ ]:
preds = torch.nn.functional.softmax(torch.Tensor(trainer.predict(test_ds).predictions)).numpy()

In [ ]:
def convert_pytorch_indices_to_scikitlearn(y):
    # [0, 1, 0] -> [[0,1],[0,1],[1,0]
    y_new = np.zeros((y.shape[0], 2))
    y_new[y == 1, 1] = 1
    y_new[y == 0, 0] = 1
    return y_new

In [ ]:
false_positive_rate = dict()
true_positive_rate = dict()
roc_auc = dict()
for i in range(num_labels):
    (
        false_positive_rate[i],
        true_positive_rate[i],
        _,
    ) = sklearn.metrics.roc_curve(
        convert_pytorch_indices_to_scikitlearn(df_reduced.iloc[indices_test].label.values)[:, i],
        preds[:, i],
    )
    roc_auc[i] = sklearn.metrics.auc(false_positive_rate[i], true_positive_rate[i])

In [ ]:
plt.figure()
lw = 2
for i in range(num_labels):
    plt.plot(
        false_positive_rate[i],
        true_positive_rate[i],
        lw=lw,
        label="ROC curve (area = %0.2f) for %i" % (roc_auc[i], i),
    )

plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()

In [ ]:
print(
    sklearn.metrics.classification_report(
        df_reduced.iloc[indices_test].label.values,
        preds.argmax(-1),
        target_names=["emotional", "neutral"],
        digits=4,
    )
)

In [ ]:
cm = sklearn.metrics.confusion_matrix(df_reduced.iloc[indices_test].label.values, preds.argmax(-1))
disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["emotional", "neutral"])
disp.plot()
ax = plt.gca()
ax.tick_params(axis="x", labelrotation=45)